In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_fake=pd.read_csv("Fake.csv")
df_true=pd.read_csv("True.csv")

In [3]:
df_fake["class"] = 0
df_true["class"] = 1

In [4]:
df_marge = pd.concat([df_fake, df_true], axis =0 )

In [5]:
df = df_marge.drop(["title", "subject","date"], axis = 1)

In [6]:
df = df.sample(frac = 1)

In [7]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [8]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [9]:
df["text"] = df["text"].apply(wordopt)

In [10]:
x = df["text"]
y = df["class"]

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [12]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# LOGISTIC REGRESSION

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [15]:
pred_lr=LR.predict(xv_test)

In [16]:
sc=LR.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * sc))
print(classification_report(y_test, pred_lr))

Test score: 98.69 %
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5916
           1       0.98      0.99      0.99      5309

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [17]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)

    return print("\n\nLR Prediction: {} ".format(output_lable(pred_LR[0]),))


In [18]:
news = str(input())
manual_testing(news)

President Joe Biden stepped up the pressure on Israel to end 10 days of violent skirmishes with Palestinians, making clear in a call Wednesday with Prime Minister Benjamin Netanyahu that he expected “significant de-escalation” by day’s end.  Biden asked Netanyahu to move “toward the path to a cease-fire,” according to a White House description of their conversation.  There is pressure, too, on Biden to do more, with more than 200 people killed in the fighting. Until Wednesday, Biden had avoided pushing the American ally more directly and publicly for a cease-fire or conveyed such a level of urgency for ending Israeli airstrikes targeting Hamas in the thickly populated Gaza Strip.  The Biden administration had relied on what officials described as “quiet, intensive” diplomacy, including quashing a U.N. Security Council statement that would have addressed a cease-fire. The administration’s handling opened a divide between Biden and Democratic lawmakers, dozens of whom have called for a c

In [19]:
import pickle
pickle.dump(LR,open('model.pkl', 'wb'))

In [20]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))
score = loaded_model.score(xv_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

def model_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR1 = loaded_model.predict(new_xv_test)

    return print("\n\nModel Prediction: {} ".format(output_lable(pred_LR1[0]),))

Test score: 98.69 %


In [21]:
news = str(input())
model_testing(news)

President Joe Biden stepped up the pressure on Israel to end 10 days of violent skirmishes with Palestinians, making clear in a call Wednesday with Prime Minister Benjamin Netanyahu that he expected “significant de-escalation” by day’s end.  Biden asked Netanyahu to move “toward the path to a cease-fire,” according to a White House description of their conversation.  There is pressure, too, on Biden to do more, with more than 200 people killed in the fighting. Until Wednesday, Biden had avoided pushing the American ally more directly and publicly for a cease-fire or conveyed such a level of urgency for ending Israeli airstrikes targeting Hamas in the thickly populated Gaza Strip.  The Biden administration had relied on what officials described as “quiet, intensive” diplomacy, including quashing a U.N. Security Council statement that would have addressed a cease-fire. The administration’s handling opened a divide between Biden and Democratic lawmakers, dozens of whom have called for a c